In [10]:
from qiskit import (QuantumCircuit,execute,Aer)
from qiskit.visualization import plot_histogram
from qiskit import*
from django.db import models
from docplex.mp.model import Model                                                                                                      
import docplex
import numpy as np
from itertools import permutations
import gzip
import pandas as pd
# import qiskit.optimization.ising.tsp.TspData as tp
from qiskit.optimization.ising.tsp import TspData
# from qiskit.aqua.optimization import TspData
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
import time
from qiskit import BasicAer
from qiskit.tools.visualization import plot_histogram
from qiskit.optimization.ising import tsp
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.algorithms import QAOA, ExactEigensolver

from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance
from qiskit.optimization.ising.common import sample_most_likely
from qiskit.aqua.components.optimizers import COBYLA

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
from qiskit import IBMQ

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute
from qiskit.tools.jupyter import *
from qiskit.providers.ibmq import least_busy

In [11]:
from qiskit import IBMQ
IBMQ.save_account('faf8747c025d9895e5d8273432c97ba5fb82e352200b4a153707adff0a5620e06b1f760fbf9a3a9988486f23698bc537c5e9d82979ee4ba699618a43eee1130d',overwrite = True)

In [12]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [13]:
G = []

In [14]:
p = [4,5,6,7,8,9,10,11]
q = [i**(2) for i in p ]
m = 0
v = open("Matrices.txt"  , "r")
w = v.read().split()
for i in range (len(w)):
    w[i] = int(float(w[i]))
for i in range (len(q)):
    G.append(np.reshape(w[m:m+q[i]] , (p[i] , p[i])))
    m = m + q[i]
print('Completed Successfully')

Completed Successfully


In [4]:
#Using Aer Backends
def VQE1(n, qubitOp,offset, s):
    seed = 10598
    start = time.time()
    spsa = SPSA(max_trials=n)
    ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
    vqe =  VQE(qubitOp, ry, spsa)

    backend = BasicAer.get_backend('statevector_simulator')

    quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed,shots = s)

    result = vqe.run(quantum_instance)
    end = time.time()
    t = end - start
    x = sample_most_likely(result['eigvecs'][0])

    return  tsp.tsp_feasible(x) , result , t

In [7]:
x = TspData('tmp',len(G[0]),np.zeros((3,3)),G[0])
qubitOp ,offset = tsp.get_operator(x)
VQE1(120, qubitOp,offset, 700)

In [ ]:
#Grid Search VQE 
l = np.arange(100,300,20) #max trails
s = np.arange(100, 1024 , 200) # shots
for k in range(1):
    f = []
    for i in range(len(l)):
        for j in range(len(s)):
            h = G[0]
            x = TspData('tmp',len(h),np.zeros((3,3)),h)
            qubitOp ,offset = tsp.get_operator(x)
            a = VQE1(int(l[i]), qubitOp,offset, s[j])
            f.append((a, l[i] , s[j]))
            print('Iteration:',i,j)
    p = 0
    for m in f:
        if m[0] == True and p != -1:
            print('Feasible')
            D[k,0] = len(V[k])
            #max trails
            D[k,1] = m[1]
            #shots
            D[k,2] = m[2]
            p = -1
        else:
            print('Infeasible')
print('Successfully Complete')

In [8]:
#Using actual quantum device
def VQE2(n,qubitOp,offset,s):
    seed = 10598
    f = time.clock()
    spsa = SPSA(max_trials = n)
    ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
    vqe = VQE(qubitOp, ry, spsa)
    my_provider = IBMQ.get_provider('ibm-q-wits')
    my_provider.backends()
    device = my_provider.get_backend('ibmq_cambridge')

    quantum_instance = QuantumInstance(device, seed_simulator=seed, seed_transpiler=seed , skip_qobj_validation = False,shots = s )

    result = vqe.run(quantum_instance)
    g = time.clock()
    
    return

In [16]:
x = TspData('tmp',len(G[0]),np.zeros((3,3)),G[0])
qubitOp ,offset = tsp.get_operator(x)
VQE2(120, qubitOp,offset, 700)